## This notebook aims to create a set of output layers in the hunt for springs.  

##FIXME

- it uses WOfS and tasselled cap wetness to look at potential location and seasonality of springs
 - it outputs a layer of water bodies identified using WOFS where water is present >90% of obs 
 - it outputs a layer intersecting with major rivers
 - it then calculates tasselled cap wetness in the same area, and calculates either overthreshold wetness or percentiles, I haven't decided yet. 
 

In [ ]:
# 